In [1]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiaoxiaoren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiaoxiaoren/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
course = pd.read_csv('courseNew.csv', sep="\t", encoding="ISO-8859-1")
removeIndices = [78,79,4326,20339,20340,20343,20346,20350,20351,20352,20353,20354,20356,20357,20360,20365,20366,20368,20371,20373,20374,20376,20378,20380,20382,20383,20391,20393,20394,20395,20396,20397,20399,20404,20405,20407,20408,20409,20411,20412,20413,20415,20416,20417,20425,20427,20431,20432,20433,20434,20435,20436,20437,20438,20440,20447,20449,20450,20453,20455,20457,20460,20462,20465,20466,20467,20468,20469,20470,20472,20473,20474,20475,20476,20477,20478,20479,20480,20481,20482,20484,20486,20487,20488,20489,20490,20492,20493,20495,20497,20498,20499,20500,20501,20502,20505,20507,20509,20511,20512,20513,20514,20515,20516,20517,20518,20519,20526,20530,20532,20534,20535,20537,20538,20541,20545,20546,20548,20551,20554,20555,20560,20561,20564,20566,20570,20578,20579,20580,20583,20586,20588,20599,20600,20602,20603,20604,20606,20607,20608,20610,20611,20625,20626,20644,20645,20646,20654,20659,20666,20671,20675,20681,20684,20686,20690,20691,20697,20704,20705,20706,20707,20710,20711,20714]

for index in removeIndices:
    course = course.drop(index)

course.head()

,course_name,course_description
0,Creating and Capturing Mixed Reality,Creating for Mixed Reality involves a variety ...
1,Extracting Information From Music Signals,The course introduces audio signal processing ...
2,Introduction à léconométrie,L'analyse de données quantitatives est devenue...
3,Introduction to Corporate Finance,This course will provide a general understandi...
4,Mobile Healthcare technologies for patients a...,Introduction to Biomedical and Health Informat...


In [3]:
course.shape

(20553, 2)

In [4]:
course = course.reset_index()
course.head()

,index,course_name,course_description
0,0,Creating and Capturing Mixed Reality,Creating for Mixed Reality involves a variety ...
1,1,Extracting Information From Music Signals,The course introduces audio signal processing ...
2,2,Introduction à léconométrie,L'analyse de données quantitatives est devenue...
3,3,Introduction to Corporate Finance,This course will provide a general understandi...
4,4,Mobile Healthcare technologies for patients a...,Introduction to Biomedical and Health Informat...


In [5]:
course["course_info"] = course["course_name"] + " " + course["course_description"]
course = course.drop(columns=['course_description'])
course.head()

,index,course_name,course_info
0,0,Creating and Capturing Mixed Reality,Creating and Capturing Mixed Reality Creating...
1,1,Extracting Information From Music Signals,Extracting Information From Music Signals Th...
2,2,Introduction à léconométrie,Introduction à léconométrie L'analyse de don...
3,3,Introduction to Corporate Finance,Introduction to Corporate Finance This course...
4,4,Mobile Healthcare technologies for patients a...,Mobile Healthcare technologies for patients a...


In [6]:
course["course_info"] = course['course_info'].str.lower()
course.head()

,index,course_name,course_info
0,0,Creating and Capturing Mixed Reality,creating and capturing mixed reality creating...
1,1,Extracting Information From Music Signals,extracting information from music signals th...
2,2,Introduction à léconométrie,introduction à léconométrie l'analyse de don...
3,3,Introduction to Corporate Finance,introduction to corporate finance this course...
4,4,Mobile Healthcare technologies for patients a...,mobile healthcare technologies for patients a...


In [7]:
course['course_name'] = course['course_name'].astype(str)
course['course_info'] = course['course_info'].astype(str)

In [8]:
for i in range(len(course)):
    #print(i, course.loc[i, "course_info"]);
    course.loc[i, "course_info"] = re.sub(r'[^a-zA-Z ]', '', course.loc[i, "course_info"])

In [9]:
course.head()

,index,course_name,course_info
0,0,Creating and Capturing Mixed Reality,creating and capturing mixed reality creating...
1,1,Extracting Information From Music Signals,extracting information from music signals th...
2,2,Introduction à léconométrie,introduction lconomtrie lanalyse de donnes q...
3,3,Introduction to Corporate Finance,introduction to corporate finance this course...
4,4,Mobile Healthcare technologies for patients a...,mobile healthcare technologies for patients a...


In [10]:
course['word tokenize']= course["course_info"].apply(word_tokenize) 

In [11]:
course.head()

,index,course_name,course_info,word tokenize
0,0,Creating and Capturing Mixed Reality,creating and capturing mixed reality creating...,"[creating, and, capturing, mixed, reality, cre..."
1,1,Extracting Information From Music Signals,extracting information from music signals th...,"[extracting, information, from, music, signals..."
2,2,Introduction à léconométrie,introduction lconomtrie lanalyse de donnes q...,"[introduction, lconomtrie, lanalyse, de, donne..."
3,3,Introduction to Corporate Finance,introduction to corporate finance this course...,"[introduction, to, corporate, finance, this, c..."
4,4,Mobile Healthcare technologies for patients a...,mobile healthcare technologies for patients a...,"[mobile, healthcare, technologies, for, patien..."


In [12]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(tokenizedWord):
    return [lemmatizer.lemmatize(w) for w in tokenizedWord]

course['word_tokenize'] = course['word tokenize'].apply(lemmatize_text)

In [13]:
stemmer = SnowballStemmer("english")
course['word_tokenize'] = course['word_tokenize'].apply(lambda x: [stemmer.stem(y) for y in x])

In [14]:
course.head()

,index,course_name,course_info,word tokenize,word_tokenize
0,0,Creating and Capturing Mixed Reality,creating and capturing mixed reality creating...,"[creating, and, capturing, mixed, reality, cre...","[creat, and, captur, mix, realiti, creat, for,..."
1,1,Extracting Information From Music Signals,extracting information from music signals th...,"[extracting, information, from, music, signals...","[extract, inform, from, music, signal, the, co..."
2,2,Introduction à léconométrie,introduction lconomtrie lanalyse de donnes q...,"[introduction, lconomtrie, lanalyse, de, donne...","[introduct, lconomtri, lanalys, de, donn, quan..."
3,3,Introduction to Corporate Finance,introduction to corporate finance this course...,"[introduction, to, corporate, finance, this, c...","[introduct, to, corpor, financ, this, cours, w..."
4,4,Mobile Healthcare technologies for patients a...,mobile healthcare technologies for patients a...,"[mobile, healthcare, technologies, for, patien...","[mobil, healthcar, technolog, for, patient, an..."


In [15]:
course = course.drop(columns=['course_info', 'word tokenize'])
course.head()

,index,course_name,word_tokenize
0,0,Creating and Capturing Mixed Reality,"[creat, and, captur, mix, realiti, creat, for,..."
1,1,Extracting Information From Music Signals,"[extract, inform, from, music, signal, the, co..."
2,2,Introduction à léconométrie,"[introduct, lconomtri, lanalys, de, donn, quan..."
3,3,Introduction to Corporate Finance,"[introduct, to, corpor, financ, this, cours, w..."
4,4,Mobile Healthcare technologies for patients a...,"[mobil, healthcar, technolog, for, patient, an..."


In [16]:
vocabulary = set()

for doc in course['word_tokenize']:
    vocabulary.update(doc)

vocabulary = list(vocabulary)
print(len(vocabulary))

print(vocabulary)

48649
['lf', 'ovdj', 'expat', 'joan', 'cba', 'relevant', 'isla', 'bookbas', 'vial', 'cholodenko', 'roofer', 'outburst', 'abelian', 'rewritten', 'interrelacion', 'thorac', 'tftp', 'hinder', 'drawstr', 'archaeoastronomi', 'souffl', 'dem', 'bergman', 'bellini', 'bireyler', 'pretens', 'boda', 'alliant', 'gagaku', 'interna', 'sheryl', 'wrath', 'jee', 'sidetosid', 'hydrologiqu', 'daha', 'signal', 'faceless', 'snowflak', 'riley', 'hydrogen', 'fiveingredi', 'confucius', 'intraor', 'hone', 'sharp', 'exposur', 'mythic', 'nanodegreehttpswwwudacitycomcourseiosdevelopernanodegreend', 'frn', 'verstehen', 'certificado', 'unavoid', 'andat', 'allez', 'creep', 'fullscal', 'mcp', 'vive', 'telegraph', 'kimberley', 'productiva', 'piper', 'jogo', 'adelantar', 'rivalutar', 'thunderstorm', 'bloodstream', 'zugnglichen', 'messag', 'accept', 'endtoend', 'streamlin', 'interrogano', 'inseam', 'mouseov', 'unearth', 'fred', 'vincenzo', 'detta', 'arithmtiqu', 'tornandoo', 'bancair', 'nanc', 'cantin', 'needless', 'fed

In [17]:
course['word_tokenize'] = course['word_tokenize'].apply(', '.join)
course.head()

,index,course_name,word_tokenize
0,0,Creating and Capturing Mixed Reality,"creat, and, captur, mix, realiti, creat, for, ..."
1,1,Extracting Information From Music Signals,"extract, inform, from, music, signal, the, cou..."
2,2,Introduction à léconométrie,"introduct, lconomtri, lanalys, de, donn, quant..."
3,3,Introduction to Corporate Finance,"introduct, to, corpor, financ, this, cours, wi..."
4,4,Mobile Healthcare technologies for patients a...,"mobil, healthcar, technolog, for, patient, and..."


In [18]:
course.to_csv('coursePreprocessed.csv', index = False)

In [19]:
course = pd.read_csv('coursePreprocessed.csv')

In [20]:
course.head()

,index,course_name,word_tokenize
0,0,Creating and Capturing Mixed Reality,"creat, and, captur, mix, realiti, creat, for, ..."
1,1,Extracting Information From Music Signals,"extract, inform, from, music, signal, the, cou..."
2,2,Introduction à léconométrie,"introduct, lconomtri, lanalys, de, donn, quant..."
3,3,Introduction to Corporate Finance,"introduct, to, corpor, financ, this, cours, wi..."
4,4,Mobile Healthcare technologies for patients a...,"mobil, healthcar, technolog, for, patient, and..."


In [21]:
course['course_name'] = course['course_name'].astype(str)
course['word_tokenize'] = course['word_tokenize'].astype(str)

In [22]:
# vocabulary = set()

# for doc in course['word_tokenize']:
#     vocabulary.update(doc.split(','))

# vocabulary = list(vocabulary)
# print(len(vocabulary))

# print(vocabulary)

In [23]:
tfidf = TfidfVectorizer(vocabulary=vocabulary)

tfidf.fit(course['word_tokenize'])

tfidf_tran=tfidf.transform(course['word_tokenize'])

In [24]:
def genQueryVector(tokens):
    query = np.zeros((len(vocabulary)))    
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            query[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return query

In [25]:
def cosineSimilarity(a, b):
    cosSim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cosSim

In [26]:
def cosine_similarity_T(k, query):
    preprocessed_query = re.sub("\W+", " ", query).strip()
    
    tokens = word_tokenize(str(preprocessed_query))
    tokens = lemmatize_text(tokens)
    tokens = [stemmer.stem(word) for word in tokens]
    tokens = ', '.join([str(word) for word in tokens]) 
        
    q_df = pd.DataFrame(columns=['queryInfo'])
    q_df.loc[0,'queryInfo'] = tokens
    
    d_cosines = []
    
    query_vector = genQueryVector(q_df['queryInfo'])
        
    for d in tfidf_tran.A:
        d_cosines.append(cosineSimilarity(query_vector, d))
        
#     print(d_cosines.sort())
    
#    print(np.argwhere(np.isnan(np.array(d_cosines))))

    out = np.array(d_cosines).argsort()[-k:][::-1]
#     print(out)
    a = pd.DataFrame()
    for i in out:
        a.loc[i,'courseName'] = course.loc[i, 'course_name'];
    return a

In [33]:
cosine_similarity_T(20, "flask python restful")

,courseName
6806,Flask Basics
2448,Build a Social Network with Flask
6807,Flask REST API
1708,Authentication & Authorization: OAuth
15305,Python for File Systems
15090,Programming in Python
15528,Real-Time Analytics with Apache Storm
6856,Fondamenti di meccanica applicata
6855,Fondamenti di Informatica
6854,Fondamenti anatomo-fisiologici dell'attività m...


In [29]:
import pickle
with open("vocabulary.txt", "wb") as fp:  
    pickle.dump(vocabulary, fp)

In [30]:
with open("vocabulary.txt", "rb") as fp: 
    v = pickle.load(fp)

In [31]:
print(len(v))

48649
